## Predicting Clinical Deterioration Using EHR Time Series

Time estimate: **20** minutes

## Objectives

After completing this lab, you will be able to:

 - Explain the structure and characteristics of time-series electronic health record (EHR) data
 - Train a Long Short-Term Memory (LSTM) neural network to predict ICU admission risk
 - Evaluate model performance and interpret results using standard classification metrics


## What you will do in this lab

In this lab, you will use a simulated time-series dataset of patient vital signs to develop and test an LSTM model.

You will:

 - Load the time-series EHR data into a dataframe
 - Explore and understand the structure of the time-series data
 - Prepare data sequences suitable for model training
 - Split the dataset into training and testing subsets
 - Build and train an LSTM neural network model
 - Evaluate model performance and generate predictions
 - Visualize results using a confusion matrix and Receiver Operating Characteristic (ROC) curve


## Overview

Clinical deterioration refers to the worsening of a patient's condition over time. Early detection of deterioration allows healthcare teams to intervene before critical events such as cardiac arrest or respiratory failure occur.

In hospitals, patients are continuously monitored with vital signs recorded at regular intervals. By analyzing these time-series patterns, machine learning models can identify subtle changes that may indicate impending deterioration.

Imagine a patient admitted to a general ward. Assume their vital signs are recorded every hour:

- Heart rate starts at 75 bpm, gradually increases to 110 bpm
- Blood pressure drops from 120/80 to 90/60
- Oxygen saturation decreases from 98% to 92%

These trends suggest the patient may need intensive care. 

The goal is to predict early whether a patient will require ICU admission based on sequential vital sign measurements.

### Why Long Short-Term Memory (LSTM)?

Traditional machine learning models such as Random Forest treat each data point independently. They cannot capture temporal dependencies, which is how one measurement relates to previous measurements over time.

LSTM is a type of Recurrent Neural Network (RNN) specifically designed for sequential data:

**Memory Cells**: LSTM can remember important patterns from earlier time steps and use them to make predictions.

**Temporal Dependencies**: It understands that vital signs at time t=5 may depend on values at t=1, t=2, t=3, t=4.

**Gradient Flow**: Unlike simple RNNs, LSTMs avoid the "vanishing gradient" problem, allowing them to learn long-term patterns.

Example: If a patient's heart rate has been steadily increasing over the past 6 hours, LSTM recognizes this trend and uses it for prediction. A traditional model would only see the current heart rate value without context.

### Assessing Accuracy and clinical utility

Accuracy: How well does the model classify patients into correct categories? (e.g., 88% correct).

Clinical utility: Is the model actually useful in practice?

Example: A model may have high accuracy but if it misses too many patients who actually need ICU care (low sensitivity), it could lead to preventable adverse outcomes.

Metrics such as Precision, Recall, Sensitivity, Specificity, and ROC-AUC help us assess whether the model is clinically valuable.

**Sensitivity (Recall)**: Of all patients who needed ICU, how many did the model identify?

**Specificity**: Of all patients who did not need ICU, how many did the model correctly classify?

**ROC-AUC**: Measures the model's ability to distinguish between ICU and non-ICU patients across different threshold settings.

## About the dataset

In this lab, you will use a simulated time-series dataset representing patient vital signs collected over time in a hospital setting.

This dataset contains sequential measurements of patient vital signs recorded at regular intervals during hospital stays. Each patient has multiple time-stamped observations, making it suitable for time-series forecasting of clinical deterioration. The goal is to predict whether a patient will require ICU admission based on their vital sign trajectories.

Column Descriptions

1. **patient_id** - Unique identifier for each patient in the hospital system.

2. **time_step** - Sequential time point representing when the measurement was taken (e.g., hourly intervals).

3. **heart_rate** - Heart rate in beats per minute (bpm). Normal range: 60-100 bpm.

4. **systolic_bp** - Systolic blood pressure in mmHg. Normal range: 90-120 mmHg.

5. **diastolic_bp** - Diastolic blood pressure in mmHg. Normal range: 60-80 mmHg.

6. **respiratory_rate** - Breathing rate in breaths per minute. Normal range: 12-20 breaths/min.

7. **temperature** - Body temperature in degrees Celsius. Normal range: 36.5-37.5°C.

8. **spo2** - Oxygen saturation percentage. Normal range: 95-100%.

9. **icu_admission** - Binary target variable (0 = No ICU admission, 1 = ICU admission required).

### Time-series characteristics

- Each patient has multiple sequential observations (typically 10-20 time steps)
- Measurements are taken at regular intervals during hospital stay
- The target (icu_admission) is assigned to the entire patient sequence
- Temporal patterns in vital signs provide predictive signals for deterioration

## Setup

For this lab, you will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for managing the data
*   [`numpy`](https://numpy.org/) for numerical operations and array manipulation
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for data preprocessing and evaluation metrics
*   [`tensorflow/keras`](https://www.tensorflow.org/) for building and training LSTM neural networks

### Installing required libraries

In [ ]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install tensorflow
!pip install matplotlib
!pip install seaborn

### Importing required libraries

In [ ]:
# Import pandas for data manipulation
import pandas as pd
# Import numpy for numerical operations
import numpy as np

# Import preprocessing tools from scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Import evaluation metrics from scikit-learn
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    roc_curve, auc, recall_score
)

# Import TensorFlow and Keras for building the LSTM model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Import matplotlib and seaborn for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("All libraries imported successfully!")
print("Ready to begin clinical deterioration prediction analysis.")

## Step 1: Load the data from a csv file into a dataframe

Since time-series vital signs data with patient_id and time_step structure is not readily available publicly, you will first generate a synthetic dataset that simulates realistic hospital vital signs monitoring.

Load the data from a csv file

In [ ]:
df = pd.read_csv("https://advanced-machine-learning-for-medical-data-8e1579.gitlab.io/labs/lab9/patient_vitals_hospital1.csv")
print(f"Total records: {df.shape[0]}")
print(f"Number of patients: {df.patient_id.nunique()}")
print(f"Time steps per patient: {df.shape[0]/df.patient_id.nunique()}")

In [ ]:
# Display first 5 rows to see the time-series structure
# Notice how each patient has multiple sequential measurements
df.head(5)

Let's find out the structure of the dataset:

In [ ]:
# Display dataset dimensions
print(f'Number of rows: {df.shape[0]}')
print(f'Number of columns: {df.shape[1]}')
print(f'\nColumns: {list(df.columns)}')

### Basic statistical information about features

In [ ]:
# Display basic statistical information about each vital sign
print("=== BASIC STATISTICS FOR ALL FEATURES ===")
df.describe()

Check the distribution of the target variable

In [ ]:
# Check class distribution
# In medical datasets, it is important to check if classes are balanced
print("ICU Admission Distribution:")
print(df.groupby('patient_id')['icu_admission'].first().value_counts())
print("\nPercentage:")
print(df.groupby('patient_id')['icu_admission'].first().value_counts(normalize=True) * 100)

## Step 2: Explore the time-series structure

Before building the model, you need to understand how the time-series data is organized. Each patient has multiple sequential measurements, and you will visualize these patterns.

In [ ]:
# Select one patient who required ICU admission
icu_patient = df[df['icu_admission'] == 1]['patient_id'].iloc[0]
icu_patient_data = df[df['patient_id'] == icu_patient]

# Select one stable patient who did not require ICU
stable_patient = df[df['icu_admission'] == 0]['patient_id'].iloc[0]
stable_patient_data = df[df['patient_id'] == stable_patient]

In [ ]:
# Visualize vital sign trends for both patients
fig, axes = plt.subplots(3, 2, figsize=(14, 10))
fig.suptitle('Comparison of Vital Sign Trends: ICU vs Stable Patient', fontsize=16)

vital_signs = ['heart_rate', 'systolic_bp', 'respiratory_rate', 'temperature', 'spo2', 'diastolic_bp']
titles = ['Heart Rate (bpm)', 'Systolic BP (mmHg)', 'Respiratory Rate (breaths/min)', 
          'Temperature (°C)', 'SpO2 (%)', 'Diastolic BP (mmHg)']

for idx, (vital, title) in enumerate(zip(vital_signs, titles)):
    row = idx // 2
    col = idx % 2
    
    axes[row, col].plot(icu_patient_data['time_step'], icu_patient_data[vital], 
                        marker='o', label='ICU Patient', color='blue')
    axes[row, col].plot(stable_patient_data['time_step'], stable_patient_data[vital], 
                        marker='s', label='Stable Patient', color='orange')
    axes[row, col].set_xlabel('Time Step (hours)')
    axes[row, col].set_ylabel(title)
    axes[row, col].legend()
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice how the ICU patient shows deteriorating trends:")
print("- Heart rate increases over time")
print("- Blood pressure decreases")
print("- Oxygen saturation (SpO2) decreases")
print("\nThe stable patient maintains relatively constant vital signs.")

## Step 3: Prepare time-series sequences

LSTM models require data in a specific 3D format: (samples, time_steps, features)

- samples: Number of patients
- time_steps: Number of sequential measurements per patient
- features: Number of vital signs measured

You will reshape the data to fit this structure.

In [ ]:
# Define the feature columns (vital signs)
feature_columns = ['heart_rate', 'systolic_bp', 'diastolic_bp', 
                   'respiratory_rate', 'temperature', 'spo2']

In [ ]:
# Extract unique patient IDs
patient_ids = df['patient_id'].unique()
print(f"Number of unique patients: {len(patient_ids)}")

In [ ]:
# Create sequences for each patient
# Each sequence contains all time steps for one patient
X_sequences = []
y_labels = []

for patient_id in patient_ids:
    # Get all measurements for this patient
    patient_data = df[df['patient_id'] == patient_id].sort_values('time_step')
    
    # Extract the vital signs as features
    sequence = patient_data[feature_columns].values
    X_sequences.append(sequence)
    
    # Extract the label (same for all time steps of a patient)
    label = patient_data['icu_admission'].iloc[0]
    y_labels.append(label)

In [ ]:
# Convert lists to numpy arrays
X = np.array(X_sequences)
y = np.array(y_labels)

print(f"Shape of X (input sequences): {X.shape}")
print(f"  - Number of patients: {X.shape[0]}")
print(f"  - Time steps per patient: {X.shape[1]}")
print(f"  - Number of features (vital signs): {X.shape[2]}")
print(f"\nShape of y (target labels): {y.shape}")

### Normalize the features

Neural networks perform better when features are normalized (scaled to similar ranges). You will use StandardScaler to standardize each vital sign.

In [ ]:
# Reshape X to 2D for scaling (samples * time_steps, features)
n_samples, n_timesteps, n_features = X.shape
X_reshaped = X.reshape(-1, n_features)

print(f"Original shape: {X.shape}")
print(f"Reshaped for scaling: {X_reshaped.shape}")

In [ ]:
# Apply StandardScaler
# This transforms each feature to have mean=0 and standard deviation=1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reshaped)

# Reshape back to 3D (samples, time_steps, features)
X_scaled = X_scaled.reshape(n_samples, n_timesteps, n_features)

print(f"Scaled data shape: {X_scaled.shape}")
print("\nFeatures have been normalized for optimal LSTM training.")

## Step 4: Split the dataset

You will split the data into training and testing sets (80:20 ratio) while maintaining the class distribution using stratification.

In [ ]:
# Split the data into train and test sets
# stratify=y ensures both sets have similar proportions of ICU vs non-ICU patients
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("Training set:")
print(f"  - Number of patients: {X_train.shape[0]}")
print(f"  - ICU admissions: {np.sum(y_train == 1)}")
print(f"  - No ICU admissions: {np.sum(y_train == 0)}")

print("\nTest set:")
print(f"  - Number of patients: {X_test.shape[0]}")
print(f"  - ICU admissions: {np.sum(y_test == 1)}")
print(f"  - No ICU admissions: {np.sum(y_test == 0)}")

## Step 5: Build and train an LSTM model

Now you will build a deep learning model using LSTM layers. LSTM networks are designed to learn patterns in sequential data.

### Understand the LSTM architecture

The model consists of:

1. **LSTM Layer 1**: Processes the sequence of vital signs, learning temporal patterns. Returns sequences to the next layer.
   - 64 units (neurons)
   - return_sequences=True: passes output to the next LSTM layer

2. **Dropout Layer 1**: Randomly drops 30% of connections during training to prevent overfitting

3. **LSTM Layer 2**: Further processes the temporal patterns
   - 32 units
   - return_sequences=False: only outputs the final time step

4. **Dropout Layer 2**: Another dropout layer for regularization

5. **Dense Layer**: Fully connected layer that produces the final prediction
   - 1 unit with sigmoid activation
   - Outputs probability of ICU admission (0 to 1)

In [ ]:
# Build the LSTM model
model = Sequential()

# First LSTM layer with 64 units
# return_sequences=True means it will pass sequences to the next LSTM layer
model.add(LSTM(units=64, return_sequences=True, input_shape=(n_timesteps, n_features)))

# Dropout layer to prevent overfitting
# Randomly sets 30% of inputs to 0 during training
model.add(Dropout(0.3))

# Second LSTM layer with 32 units
# return_sequences=False means it will only output the final time step
model.add(LSTM(units=32, return_sequences=False))

# Another dropout layer
model.add(Dropout(0.3))

# Dense output layer with sigmoid activation for binary classification
# Outputs a probability between 0 and 1
model.add(Dense(units=1, activation='sigmoid'))

print("LSTM model architecture created successfully!")

In [ ]:
# Display model summary
# This shows the architecture, number of parameters, and output shapes
model.summary()

### Compile the model

Before training, you need to compile the model by specifying:
- **Optimizer**: Adam (adaptive learning rate optimizer)
- **Loss function**: Binary crossentropy (for binary classification)
- **Metrics**: Accuracy (to monitor during training)

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer with learning rate 0.001
    loss='binary_crossentropy',            # Loss function for binary classification
    metrics=['accuracy']                   # Track accuracy during training
)

print("Model compiled and ready for training!")

### Train the model

Now you will train the model on the training data. The model will learn to recognize patterns in vital sign sequences that indicate ICU admission risk.

In [ ]:
# Train the model
# epochs: number of times the model sees the entire training dataset
# batch_size: number of samples processed before updating model weights
# validation_split: use 20% of training data for validation during training
# verbose: 1 shows progress bar, 0 is silent

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print("\nModel training completed!")

### Visualize training history

Plotting the training and validation accuracy/loss helps you understand how well the model learned and whether it is overfitting.

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
axes[0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Accuracy Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot loss
axes[1].plot(history.history['loss'], label='Training Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Model Loss Over Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Training history visualization complete.")
print("If training and validation curves are close, the model is generalizing well.")
print("If training accuracy is much higher than validation, the model may be overfitting.")

## Step 6: Evaluate the model and make predictions

Now that the model is trained, you will evaluate its performance on the test set (unseen data).

In [ ]:
# Make predictions on test set
# model.predict returns probabilities
y_pred_proba = model.predict(X_test)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print(f"Number of predictions: {len(y_pred)}")
print(f"Predicted ICU admissions: {np.sum(y_pred == 1)}")
print(f"Predicted no ICU: {np.sum(y_pred == 0)}")

### Calculate evaluation metrics

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")
print(f"Accuracy Percentage: {accuracy * 100:.2f}%")

In [ ]:
# Display classification report
# This shows precision, recall, f1-score for each class
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=['No ICU', 'ICU Required']))

## Step 7: Display confusion matrix and ROC curve

Visual evaluation tools help you understand model performance from a clinical perspective.

### Confusion matrix

A confusion matrix shows:
- True Negatives (TN): Correctly predicted no ICU
- False Positives (FP): Incorrectly predicted ICU (false alarm)
- False Negatives (FN): Incorrectly predicted no ICU (missed case)
- True Positives (TP): Correctly predicted ICU

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
ax = sns.heatmap(cm, annot=False, fmt="d", cmap="Blues",
                 xticklabels=["No ICU", "ICU Required"],
                 yticklabels=["No ICU", "ICU Required"],
                 cbar_kws={"shrink": 0.8})

# Manually add text annotations with different font sizes
labels = [["True\nNegative", "False\nPositive"],
          ["False\nNegative", "True\nPositive"]]

for i in range(2):
    for j in range(2):
        # Add the number with larger font
        ax.text(j + 0.5, i + 0.4, str(cm[i, j]),
               ha="center", va="center",
               color="orange", fontsize=24, fontweight='bold')
        
        # Add the label with smaller font
        ax.text(j + 0.5, i + 0.7, labels[i][j],
               ha="center", va="top",
               color="orange", fontsize=11)

plt.title("Confusion Matrix", fontsize=14, fontweight='bold')
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("Actual", fontsize=12)
plt.tight_layout()
plt.show()

print("\nConfusion Matrix Interpretation:")
print(f"True Negatives (correctly predicted no ICU): {cm[0, 0]}")
print(f"False Positives (incorrectly predicted ICU): {cm[0, 1]}")
print(f"False Negatives (missed ICU cases): {cm[1, 0]}")
print(f"True Positives (correctly predicted ICU): {cm[1, 1]}")

### ROC curve and AUC

The Receiver Operating Characteristic (ROC) curve shows the trade-off between sensitivity and specificity at different threshold values.

AUC (Area Under Curve) summarizes the ROC curve:
- AUC = 1.0: Perfect model
- AUC = 0.5: Random guessing
- AUC > 0.8: Good model for clinical use

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12)
plt.title('ROC Curve for ICU Admission Prediction', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nROC-AUC Score: {roc_auc:.4f}")
if roc_auc > 0.9:
    print("Excellent model performance!")
elif roc_auc > 0.8:
    print("Good model performance for clinical use.")
elif roc_auc > 0.7:
    print("Acceptable model performance.")
else:
    print("Model may need improvement for clinical deployment.")

### Calculate Sensitivity and Specificity

In [ ]:
# Sensitivity (Recall) - ability to identify patients who need ICU
sensitivity = recall_score(y_test, y_pred, pos_label=1)
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Sensitivity Percentage: {sensitivity * 100:.2f}%")
print("Sensitivity measures the model's ability to correctly identify patients who require ICU admission.")

In [ ]:
# Specificity - ability to identify patients who do not need ICU
specificity = recall_score(y_test, y_pred, pos_label=0)
print(f"\nSpecificity: {specificity:.4f}")
print(f"Specificity Percentage: {specificity * 100:.2f}%")
print("Specificity measures the model's ability to correctly identify patients who do not require ICU admission.")

### Clinical interpretation

Understanding what these metrics mean in a healthcare context:

In [ ]:
print("=== Clinical Interpretation ===")
print(f"\nOut of {np.sum(y_test == 1)} patients who actually required ICU:")
print(f"  - The model correctly identified: {cm[1, 1]} patients ({sensitivity*100:.1f}%)")
print(f"  - The model missed: {cm[1, 0]} patients")

print(f"\nOut of {np.sum(y_test == 0)} patients who did not require ICU:")
print(f"  - The model correctly identified: {cm[0, 0]} patients ({specificity*100:.1f}%)")
print(f"  - The model incorrectly flagged: {cm[0, 1]} patients")

print("\nClinical Decision Making:")
if sensitivity > 0.85:
    print("High sensitivity: The model catches most patients who need ICU care.")
else:
    print("Consider adjusting the threshold to improve sensitivity and catch more at-risk patients.")

if specificity > 0.85:
    print("High specificity: The model avoids unnecessary ICU alerts for stable patients.")
else:
    print("Some false positives occur, which may lead to unnecessary resource allocation.")

# Exercises

Now, you will practice building an LSTM model on the "patient_vitals_hospital2.csv" dataset (https://advanced-machine-learning-for-medical-data-8e1579.gitlab.io/labs/lab9/patient_vitals_hospital2.csv) and use different parameters.

For the exercises, you will create a new synthetic dataset with slightly different characteristics to practice the entire workflow.

### Exercise 1: Loading a different dataset

In [ ]:
# your code goes here


<details>
    <summary>Click here for a hint</summary>
    
Refer to Step 1, and use "patient_vitals_hospital2.csv"

</details>

<details>
    <summary>Click here for solution</summary>

```python
df_ex = pd.read_csv("https://advanced-machine-learning-for-medical-data-8e1579.gitlab.io/labs/lab9/patient_vitals_hospital2.csv")
print(f"Total records: {df.shape[0]}")
print(f"Number of patients: {df.patient_id.nunique()}")
print(f"Time steps per patient: {df.shape[0]/df.patient_id.nunique()}")
```

</details>

### Exercise 2: Verify columns and create sequences

Verify all columns in the dataset, then create sequences (X) and labels (y). Normalize the sequences using StandardScaler. Display the shapes.

In [ ]:
# your code goes here


<details>
    <summary>Click here for a hint</summary>
    
Refer to Step 3. Use the same feature columns and follow the sequence creation and scaling steps.

</details>

<details>
    <summary>Click here for solution</summary>

```python
# Verify columns
print("Columns in dataset:")
print(df_ex.columns.tolist())

# Define features
feature_columns_ex = ['heart_rate', 'systolic_bp', 'diastolic_bp', 
                      'respiratory_rate', 'temperature', 'spo2']

# Create sequences
patient_ids_ex = df_ex['patient_id'].unique()
X_sequences_ex = []
y_labels_ex = []

for patient_id in patient_ids_ex:
    patient_data = df_ex[df_ex['patient_id'] == patient_id].sort_values('time_step')
    sequence = patient_data[feature_columns_ex].values
    X_sequences_ex.append(sequence)
    label = patient_data['icu_admission'].iloc[0]
    y_labels_ex.append(label)

X_ex = np.array(X_sequences_ex)
y_ex = np.array(y_labels_ex)

# Normalize
n_samples_ex, n_timesteps_ex, n_features_ex = X_ex.shape
X_reshaped_ex = X_ex.reshape(-1, n_features_ex)

scaler_ex = StandardScaler()
X_scaled_ex = scaler_ex.fit_transform(X_reshaped_ex)
X_scaled_ex = X_scaled_ex.reshape(n_samples_ex, n_timesteps_ex, n_features_ex)

print(f"\nX shape: {X_scaled_ex.shape}")
print(f"y shape: {y_ex.shape}")
```

</details>

### Exercise 3: Split the dataset

Split the dataset into training and testing sets (80:20 ratio) with stratification. Display the number of samples in each set and the class distribution.

In [ ]:
# insert your code here


<details>
    <summary>Click here for a hint</summary>
    
Refer to Step 4. Use train_test_split with test_size=0.2, random_state=42, and stratify=y_ex

</details>

<details>
    <summary>Click here for solution</summary>

```python
# Split the data
X_train_ex, X_test_ex, y_train_ex, y_test_ex = train_test_split(
    X_scaled_ex, y_ex, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_ex
)

print("Training set:")
print(f"  Number of patients: {X_train_ex.shape[0]}")
print(f"  ICU admissions: {np.sum(y_train_ex == 1)}")
print(f"  No ICU admissions: {np.sum(y_train_ex == 0)}")

print("\nTest set:")
print(f"  Number of patients: {X_test_ex.shape[0]}")
print(f"  ICU admissions: {np.sum(y_test_ex == 1)}")
print(f"  No ICU admissions: {np.sum(y_test_ex == 0)}")
```

</details>

### Exercise 4: Build and train LSTM model

Build an LSTM model with the following architecture:
- First LSTM layer: 128 units, return_sequences=True
- Dropout: 0.4
- Second LSTM layer: 64 units, return_sequences=False
- Dropout: 0.4
- Dense output layer: 1 unit, sigmoid activation

Compile with Adam optimizer (learning_rate=0.001) and train for 40 epochs with batch_size=16.

In [ ]:
# your code goes here


<details>
    <summary>Click here for a hint</summary>
    
Refer to Step 5. Use Sequential model and add layers in order. Remember to use input_shape for the first layer.

</details>

<details>
    <summary>Click here for solution</summary>

```python
# Build model
model_ex = Sequential()
model_ex.add(LSTM(units=128, return_sequences=True, 
                  input_shape=(n_timesteps_ex, n_features_ex)))
model_ex.add(Dropout(0.4))
model_ex.add(LSTM(units=64, return_sequences=False))
model_ex.add(Dropout(0.4))
model_ex.add(Dense(units=1, activation='sigmoid'))

# Compile
model_ex.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model architecture:")
model_ex.summary()

# Train
history_ex = model_ex.fit(
    X_train_ex, y_train_ex,
    epochs=40,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)

print("\nTraining completed!")
```

</details>

### Exercise 5: Make predictions

Use the trained model to make predictions on the test set. Convert probabilities to binary predictions using a threshold of 0.5.

In [ ]:
# your code goes here


<details>
    <summary>Click here for a hint</summary>
    
Use model.predict() to get probabilities, then apply threshold > 0.5 to convert to binary predictions.

</details>

<details>
    <summary>Click here for solution</summary>

```python
# Make predictions
y_pred_proba_ex = model_ex.predict(X_test_ex)
y_pred_ex = (y_pred_proba_ex > 0.5).astype(int).flatten()

print(f"Number of predictions: {len(y_pred_ex)}")
print(f"Predicted ICU admissions: {np.sum(y_pred_ex == 1)}")
print(f"Predicted no ICU: {np.sum(y_pred_ex == 0)}")
```

</details>

### Exercise 6: Calculate Accuracy and display classification report

Calculate the model accuracy and display the classification report.

In [ ]:
# your code goes here


<details>
    <summary>Click here for a hint</summary>
    
Use accuracy_score() and classification_report() from sklearn.metrics

</details>

<details>
    <summary>Click here for solution</summary>

```python
# Calculate accuracy
accuracy_ex = accuracy_score(y_test_ex, y_pred_ex)
print(f"Model Accuracy: {accuracy_ex:.4f}")
print(f"Accuracy Percentage: {accuracy_ex * 100:.2f}%")

# Classification report
print("\n=== Classification Report ===")
print(classification_report(y_test_ex, y_pred_ex, 
                          target_names=['No ICU', 'ICU Required']))
```

</details>

### Exercise 7: Print Sensitivity and Specificity

Calculate and display sensitivity and specificity as percentages.

In [ ]:
# your code goes here


<details>
    <summary>Click here for a hint</summary>
    
Use recall_score() with pos_label=1 for sensitivity and pos_label=0 for specificity

</details>

<details>
    <summary>Click here for solution</summary>

```python
# Sensitivity (Recall for ICU class)
sensitivity_ex = recall_score(y_test_ex, y_pred_ex, pos_label=1)
print(f"Sensitivity (Recall): {sensitivity_ex:.4f}")
print(f"Sensitivity Percentage: {sensitivity_ex * 100:.2f}%")
print("This measures the ability to identify patients who require ICU admission.")

# Specificity (Recall for No ICU class)
specificity_ex = recall_score(y_test_ex, y_pred_ex, pos_label=0)
print(f"\nSpecificity: {specificity_ex:.4f}")
print(f"Specificity Percentage: {specificity_ex * 100:.2f}%")
print("This measures the ability to identify patients who do not require ICU admission.")
```

</details>

# Congratulations! 
You have completed this lab on predicting clinical deterioration using LSTM networks. You now know how to prepare time-series healthcare data, train an LSTM model, and evaluate its performance using key metrics and visualizations.

## Authors

Ramesh Sannareddy

Copyright © 2025 SkillUp. All rights reserved.